In [1]:
!pip install sklearn --upgrade
!pip install joblib
!pip install imbalanced-learn --upgrade

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import (RandomOverSampler, SMOTE, ADASYN)
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler

Requirement already up-to-date: sklearn in c:\users\joaog\anaconda3\lib\site-packages (0.0)
Requirement already up-to-date: imbalanced-learn in c:\users\joaog\anaconda3\lib\site-packages (0.6.2)


Using TensorFlow backend.


In [2]:
nd = pd.read_csv("data.csv")
numeric_data = nd.drop(columns="Unnamed: 32")
numeric_data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [3]:
numeric_data['diagnosis_num'] =  numeric_data['diagnosis'].apply(lambda x: 0 if x == 'B' else 1)
numeric_data.columns
numeric_data = numeric_data[['id', 'diagnosis', 'diagnosis_num', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst']]

numeric_data.head()

,id,diagnosis,diagnosis_num,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [4]:
numeric_only = numeric_data.drop('diagnosis', axis=1)
numeric_df = pd.DataFrame(numeric_only)
numeric_df.head()

,id,diagnosis_num,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [5]:
target = numeric_df['diagnosis_num']
data = numeric_df.drop('diagnosis_num', axis=1)
features = data.columns

In [6]:
# EDA 
total_count_tumors = len(numeric_data)
print(f'Total Tumors on data = {total_count_tumors}')
total_count_benign = (numeric_data.diagnosis == 'B').sum()
print(f'Total Benign Tumors on data = {total_count_benign}')
total_count_malignant = (numeric_data.diagnosis == 'M').sum()
print(f'Total Malignant Tumors on data = {total_count_malignant}')
print("-----")
percent_benign = (total_count_benign/total_count_tumors)*100.00
percent_malignant = (total_count_malignant/total_count_tumors)*100.00
print(f"Percent Benign = {percent_benign}")
print(f"Percent Malignant = {percent_malignant}")

Total Tumors on data = 569
Total Benign Tumors on data = 357
Total Malignant Tumors on data = 212
-----
Percent Benign = 62.741652021089635
Percent Malignant = 37.258347978910365


# Features Selection

Select the top 10 features according to the ```feature_selection``` from ```sklearn```

In [7]:
bestfeatures = SelectKBest(k=31)
fit = bestfeatures.fit(data, target)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(data.columns)
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Feature','Score']
print(featureScores.nlargest(10,'Score'))

                 Feature       Score
28  concave points_worst  964.385393
23       perimeter_worst  897.944219
8    concave points_mean  861.676020
21          radius_worst  860.781707
3         perimeter_mean  697.235272
24            area_worst  661.600206
1            radius_mean  646.981021
4              area_mean  573.060747
7         concavity_mean  533.793126
27       concavity_worst  436.691939


In [8]:
#select the top 10 features
features_list_df = numeric_data[["concave points_worst",
"perimeter_worst",
"concave points_mean",
"radius_worst",
"perimeter_mean",
"area_worst",
"radius_mean",
"area_mean",
"concavity_mean",
"concavity_worst"]]

## OVERSAMPLING THE DATA

In [9]:
# spliting the data on train and test data
X_train, X_test, y_train, y_test = train_test_split (features_list_df, target, test_size = 0.30, random_state=21)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(398, 10) (398,)
(171, 10) (171,)


In [10]:
# Count the training data
total_count_tumors = len(y_train)
print(f'Total Tumors on train data = {total_count_tumors}')
total_count_benign = (y_train == 0).sum()
print(f'Total Benign Tumors on train data = {total_count_benign}')
total_count_malignant = (y_train == 1).sum()
print(f'Total Malignant Tumors on train data = {total_count_malignant}')
print("-----")
percent_benign = (total_count_benign/total_count_tumors)*100.00
percent_malignant = (total_count_malignant/total_count_tumors)*100.00
print(percent_benign)
print(percent_malignant)

Total Tumors on train data = 398
Total Benign Tumors on train data = 250
Total Malignant Tumors on train data = 148
-----
62.8140703517588
37.185929648241206


In [11]:
print('Original train dataset shape %s' % Counter(y_train))

Original train dataset shape Counter({0: 250, 1: 148})


#### Apply oversampling models (RandomOverSampler, SMOTE, ADASYN)

In [12]:
# Random Over Sampler Model
ros = RandomOverSampler(random_state=21)
X_ros, y_ros = ros.fit_resample(X_train, y_train)
print('Resampled RandomOverSampler train dataset shape %s' % Counter(y_ros))
print (X_ros.shape, y_ros.shape)

Resampled RandomOverSampler train dataset shape Counter({0: 250, 1: 250})
(500, 10) (500,)


In [13]:
# SMOTE: Synthetic Minority Oversampling Technique Model
sm = SMOTE(random_state=21)
X_sm, y_sm = sm.fit_resample(X_train, y_train)
print('Resampled SMOTE train dataset shape %s' % Counter(y_sm))
print (X_sm.shape, y_sm.shape)

Resampled SMOTE train dataset shape Counter({0: 250, 1: 250})
(500, 10) (500,)


In [14]:
# ADASYN: Adaptive Synthetic Sampling Model
ada = ADASYN(random_state=21)
X_ada, y_ada = ada.fit_resample(X_train, y_train)
print('Resampled ADASYN train dataset shape %s' % Counter(y_ada))
print (X_ada.shape, y_ada.shape)

Resampled ADASYN train dataset shape Counter({1: 254, 0: 250})
(504, 10) (504,)


Oversampling models comparison SMOTE VS ADASYN.

SMOTE: It finds the n-nearest neighbors in the minority class for each of the samples in the class. Then it draws a line between the the neighbors an generates random points on the lines.

ADASYN:  Works as SMOTE, but with a minor improved. After creating the sample it adds a random small values to the points. In other words instead of all the sample being linearly correlated to the parent they have a little more variance in them, been a bit scattered.

To take advantage of this smaller variation on the new data, we will use the ADASYN model to oversample the data (X_ada, y_ada).

## Pre-processing

Scale the data using the ```StandardScaler```



In [15]:
# Scale the data
X_scaler = StandardScaler().fit(X_ada)
X_train_scaled = X_scaler.transform(X_ada)
X_test_scaled = X_scaler.transform(X_test)


# Testing Models

### Model 1 - Logistic Regression

In [16]:
# import and train the model

from sklearn.model_selection import KFold, cross_val_score, validation_curve
from sklearn import linear_model

logreg = linear_model.LogisticRegression()
kfold = KFold(n_splits=5,random_state=21, shuffle=True)
cv_results = cross_val_score(logreg, X_train_scaled, y_ada, cv=kfold)
print (cv_results.mean()*100, "%")

92.05148514851486 %


In [17]:
# apply the model 

logreg.fit(X_test_scaled, y_test)
print(f"Testing Logistic Regression Data Score: {logreg.score(X_test_scaled, y_test)}")

Testing Logistic Regression Data Score: 0.9532163742690059


### Model 2 - SVC


In [18]:
# import, train and apply the model

from sklearn.svm import SVC 
from sklearn.metrics import classification_report

model_SVC = SVC()
model_SVC.fit(X_train_scaled, y_ada)
prediction_SVC = model_SVC.predict(X_test_scaled)

In [19]:
print(f"Training Data SVC Score: {model_SVC.score(X_train_scaled, y_ada)}")
print(f"Testing Data SVC Score: {model_SVC.score(X_test_scaled, y_test)}\n")
print(classification_report(y_test, prediction_SVC,
                            target_names=['Benign ', 'Malignant']))

Training Data SVC Score: 0.9325396825396826
Testing Data SVC Score: 0.9649122807017544

              precision    recall  f1-score   support

     Benign        0.97      0.97      0.97       107
   Malignant       0.95      0.95      0.95        64

    accuracy                           0.96       171
   macro avg       0.96      0.96      0.96       171
weighted avg       0.96      0.96      0.96       171



In [20]:
# use GridSearchCV to Hyperparameter Tuning the model

from sklearn.model_selection import GridSearchCV
param_grid_SVC = {'C': [1, 5, 10, 50, 100],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid_SVC = GridSearchCV(model_SVC, param_grid_SVC, verbose=3)

In [21]:
grid_SVC.fit(X_train_scaled, y_ada)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.871, total=   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.871, total=   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.832, total=   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.584, total=   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.610, total=   0.0s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ................... C=1, gamma=0.0005, score=0.941, total=   0.0s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ..........

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s



[CV] C=5, gamma=0.0001 ...............................................
[CV] ................... C=5, gamma=0.0001, score=0.810, total=   0.0s
[CV] C=5, gamma=0.0005 ...............................................
[CV] ................... C=5, gamma=0.0005, score=0.911, total=   0.0s
[CV] C=5, gamma=0.0005 ...............................................
[CV] ................... C=5, gamma=0.0005, score=0.921, total=   0.0s
[CV] C=5, gamma=0.0005 ...............................................
[CV] ................... C=5, gamma=0.0005, score=0.950, total=   0.0s
[CV] C=5, gamma=0.0005 ...............................................
[CV] ................... C=5, gamma=0.0005, score=0.881, total=   0.0s
[CV] C=5, gamma=0.0005 ...............................................
[CV] ................... C=5, gamma=0.0005, score=0.860, total=   0.0s
[CV] C=5, gamma=0.001 ................................................
[CV] .................... C=5, gamma=0.001, score=0.911, total=   0.0s
[CV] 

[CV] ................. C=100, gamma=0.0001, score=0.960, total=   0.0s
[CV] C=100, gamma=0.0001 .............................................
[CV] ................. C=100, gamma=0.0001, score=0.921, total=   0.0s
[CV] C=100, gamma=0.0001 .............................................
[CV] ................. C=100, gamma=0.0001, score=0.870, total=   0.0s
[CV] C=100, gamma=0.0005 .............................................
[CV] ................. C=100, gamma=0.0005, score=0.921, total=   0.0s
[CV] C=100, gamma=0.0005 .............................................
[CV] ................. C=100, gamma=0.0005, score=0.931, total=   0.0s
[CV] C=100, gamma=0.0005 .............................................
[CV] ................. C=100, gamma=0.0005, score=0.950, total=   0.0s
[CV] C=100, gamma=0.0005 .............................................
[CV] ................. C=100, gamma=0.0005, score=0.921, total=   0.0s
[CV] C=100, gamma=0.0005 .............................................
[CV] .

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.4s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 5, 10, 50, 100],
                         'gamma': [0.0001, 0.0005, 0.001, 0.005]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [22]:
print(f"Best GridSeachCV Parameters {grid_SVC.best_params_}")
print(f"Best GridSeachCV result {grid_SVC.best_score_}")

Best GridSeachCV Parameters {'C': 100, 'gamma': 0.005}
Best GridSeachCV result 0.9304554455445544


In [23]:
# set the best SCV model
bestSVC_model = SVC(C=100, kernel='linear', gamma=0.005)

In [24]:
#Best SVC model results
bestSVC_model.fit(X_test_scaled, y_test)
print(f"Testing Data bestSVC_model Score: {bestSVC_model.score(X_test_scaled, y_test)}")

Testing Data bestSVC_model Score: 0.9473684210526315


### Model 3 - Neural Network

In [25]:
# # One-hot encoding y to run a Neural Network Model.
from keras.utils import to_categorical

y_train_categorical = to_categorical(y_ada)
y_test_categorical = to_categorical(y_test)
print(y_train_categorical)

[[1. 0.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [26]:
# import the model and create the layers

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=6, activation='relu', input_dim=10))
model.add(Dense(units=2, activation='softmax'))

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 66        
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 14        
Total params: 80
Trainable params: 80
Non-trainable params: 0
_________________________________________________________________


In [28]:
# compile and train the model

model.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=['accuracy'])
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Train on 504 samples
Epoch 1/100
504/504 - 0s - loss: 1.0519 - accuracy: 0.2877
Epoch 2/100
504/504 - 0s - loss: 0.9381 - accuracy: 0.3036
Epoch 3/100
504/504 - 0s - loss: 0.8438 - accuracy: 0.3254
Epoch 4/100
504/504 - 0s - loss: 0.7586 - accuracy: 0.3750
Epoch 5/100
504/504 - 0s - loss: 0.6807 - accuracy: 0.5238
Epoch 6/100
504/504 - 0s - loss: 0.6148 - accuracy: 0.7679
Epoch 7/100
504/504 - 0s - loss: 0.5545 - accuracy: 0.8373
Epoch 8/100
504/504 - 0s - loss: 0.5056 - accuracy: 0.8532
Epoch 9/100
504/504 - 0s - loss: 0.4631 - accuracy: 0.8591
Epoch 10/100
504/504 - 0s - loss: 0.4283 - accuracy: 0.8651
Epoch 11/100
504/504 - 0s - loss: 0.3992 - accuracy: 0.8651
Epoch 12/100
504/504 - 0s - loss: 0.3749 - accuracy: 0.8631
Epoch 13/100
504/504 - 0s - loss: 0.3552 - accuracy: 0.8611
Epoch 14/100
504/504 - 0s - loss: 0.3384 - accuracy: 0.8690
Epoch 15/100
504/504 - 0s - loss: 0.3246 - accuracy: 0.8690
Epoch 16/100
504/504 - 0s - loss: 0.3126 - accuracy: 0.8750
Epoch 17/100
504/504 - 0s - 

In [29]:
# apply the model to the test data

model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=3)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 0.1523783787300712, Accuracy: 0.9415204524993896


### Model 4 - Random Forest

In [30]:
# import the model 

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train_scaled, y_ada)

print(f"Training RandonForest Data Score: {rf.score(X_train_scaled, y_ada)}")
print(f"Testing RandonForest Data Score: {rf.score(X_test_scaled, y_test)}")

Training RandonForest Data Score: 1.0
Testing RandonForest Data Score: 0.9473684210526315


In [31]:
# use GridSearchCV to Hyperparameter Tuning the model

param_grid_rf = {'n_estimators': [250, 300, 350],'max_depth': [125, 150, 175]}
grid_rf = GridSearchCV(rf, param_grid_rf, verbose=3)

In [32]:
grid_rf.fit(X_train_scaled, y_ada)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] max_depth=125, n_estimators=250 .................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ..... max_depth=125, n_estimators=250, score=0.941, total=   0.3s
[CV] max_depth=125, n_estimators=250 .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] ..... max_depth=125, n_estimators=250, score=0.941, total=   0.3s
[CV] max_depth=125, n_estimators=250 .................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s


[CV] ..... max_depth=125, n_estimators=250, score=0.950, total=   0.2s
[CV] max_depth=125, n_estimators=250 .................................
[CV] ..... max_depth=125, n_estimators=250, score=0.931, total=   0.3s
[CV] max_depth=125, n_estimators=250 .................................
[CV] ..... max_depth=125, n_estimators=250, score=0.880, total=   0.2s
[CV] max_depth=125, n_estimators=300 .................................
[CV] ..... max_depth=125, n_estimators=300, score=0.931, total=   0.3s
[CV] max_depth=125, n_estimators=300 .................................
[CV] ..... max_depth=125, n_estimators=300, score=0.950, total=   0.3s
[CV] max_depth=125, n_estimators=300 .................................
[CV] ..... max_depth=125, n_estimators=300, score=0.950, total=   0.3s
[CV] max_depth=125, n_estimators=300 .................................
[CV] ..... max_depth=125, n_estimators=300, score=0.931, total=   0.4s
[CV] max_depth=125, n_estimators=300 .................................
[CV] .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:   13.9s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              ra

In [33]:
print(f"Best RandomForest Parameters {grid_rf.best_params_}")
print(f"Best RandomForest result {grid_rf.best_score_}")


Best RandomForest Parameters {'max_depth': 175, 'n_estimators': 250}
Best RandomForest result 0.9363960396039603


In [36]:
best_rf = RandomForestClassifier(max_depth=175, n_estimators=250)

In [37]:
best_rf.fit(X_train_scaled, y_ada)
prediction_rf = best_rf.predict(X_test_scaled)
print(f"Testing RandomForest Data Score: {best_rf.score(X_test_scaled, y_test)}\n")

print(classification_report(y_test, prediction_rf,
                            target_names=['Benign ', 'Malignant']))

Testing RandomForest Data Score: 0.9473684210526315

              precision    recall  f1-score   support

     Benign        0.96      0.95      0.96       107
   Malignant       0.92      0.94      0.93        64

    accuracy                           0.95       171
   macro avg       0.94      0.95      0.94       171
weighted avg       0.95      0.95      0.95       171

